In [ ]:
!pip install transformers sentencepiece sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=5db46443fa18108940871b5e55bc5ec1b280e393af7a8b375907cb4729e8dd64
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


**Loading required packages**

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

**Testing Languages and the forcused language code**

In [ ]:
langs = ["en", "hi", "te", "ta", "ja", "zh-CN", "fr", "es", "de", "ru", "si"]
N = len(langs)

forcused_lang_options = langs + ["all"]

forcused_lang = "si"

assert forcused_lang in forcused_lang_options

**Model loading**

In [ ]:
print(f"[INFO] Loading the encoder..")
model = SentenceTransformer('sentence-transformers/LaBSE')
print(f"[INFO] Encoder loaded..")

[INFO] Loading the encoder..


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

[INFO] Encoder loaded..


**Parallel sentences set**

In [ ]:
sentences_set1 = ["My email address is hgf.45@gmail.com", "मेरा ईमेल पता hgf.45@gmail.com है", "నా ఇమెయిల్ చిరునామా hgf.45@gmail.com",
             "எனது மின்னஞ்சல் முகவரி hgf.45@gmail.com", "私のメールアドレスはhgf.45@gmail.comです", "我的电子邮件地址是 hgf.45@gmail.com",
             "Mon adresse e-mail est hgf.45@gmail.com", "Mi dirección de correo electrónico es hgf.45@gmail.com", "Meine E-Mail-Adresse lautet hgf.45@gmail.com",
             "Мой адрес электронной почты: hgf.45@gmail.com", "මගේ ඊමේල් ලිපිනය hgf.45@gmail.com"]

sentences_set2 = ["The language we are looking for is already present in it.", "हम जिस भाषा की तलाश कर रहे हैं, वह उसमें पहले से मौजूद है।", "మనం వెతుకుతున్న భాష ఇప్పటికే ఇందులో ఉంది.",
                  "நாம் தேடும் மொழி ஏற்கனவே இதில் உள்ளது.", "探している言語は既に存在しています。", "我们正在寻找的语言已经存在于其中。",
                  "La langue que nous recherchons y est déjà présente.", "El lenguaje que buscamos ya está presente en él.", "Die Sprache, die wir suchen, ist darin bereits vorhanden.",
                  "Язык, который мы ищем, уже присутствует в нем.", "අපි සොයන භාෂාව දැනටමත් එහි ඇත."]

**Function to compare sentece embeddings**

In [ ]:
# def compare(sentences, forcused_lang="all"):
#   t0 = time.time()
#   embeddings = model.encode(sentences)
#   print(f"[INFO] Encoding time {(time.time()-t0)*1000} ms")

#   # print(embeddings)
#   print(embeddings.shape, embeddings[0].shape)

#   if forcused_lang == "all":
#       for i in range(N):
#           for j in range(i+1, N):
#               print(f"{langs[i]}: {sentences[i]}  --- {langs[j]}: {sentences[j]}  ---> {cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1))[0][0]}")

#   else:
#       for i in range(N):
#           for j in range(i+1, N):
#               if langs[j] == forcused_lang:
#                   print(f"{langs[i]}: {sentences[i]}  --- {langs[j]}: {sentences[j]}  ---> {cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1))[0][0]}")

In [ ]:
def compare(sentences, forcused_lang="all"):
    t0 = time.time()
    embeddings = model.encode(sentences, convert_to_tensor=True)
    print(f"[INFO] Encoding time {(time.time()-t0)*1000} ms")

    # print(embeddings)
    print(embeddings.shape, embeddings[0].shape)

    #Compute cosine-similarities
    cosine_scores = cos_sim(embeddings, embeddings)

    if forcused_lang == "all":
        for i in range(N):
            for j in range(i+1, N):
                print(f"{langs[i]}: {sentences[i]}  --- {langs[j]}: {sentences[j]}  ---> {cosine_scores[i][j]}")

    else:
        for i in range(N):
            for j in range(i+1, N):
                if langs[j] == forcused_lang:
                    print(f"{langs[i]}: {sentences[i]}  --- {langs[j]}: {sentences[j]}  ---> {cosine_scores[i][j]}")

In [ ]:
def compare_two(s1, s2):
    t0 = time.time()
    embeddings = model.encode([s1, s2], convert_to_tensor=True)
    # embeddings = model.encode([s1, s2])
    print(f"[INFO] Encoding time {(time.time()-t0)*1000} ms")

    # print(embeddings)
    print(embeddings.shape, embeddings[0].shape)

    print(f"{s1} -- {s2} ---> {cos_sim(embeddings[0], embeddings[1])}")
    # print(f"{s1} -- {s2} ---> {cosine_similarity(embeddings[0].reshape(1, -1), embeddings[1].reshape(1, -1))[0][0]}")


In [ ]:
# dir(model)

**Cross-lingual sentece embedding comparison**

In [ ]:
compare(sentences_set1)
print("----"*30)
compare(sentences_set2)

[INFO] Encoding time 1306.971788406372 ms
torch.Size([11, 768]) torch.Size([768])
en: My email address is hgf.45@gmail.com  --- hi: मेरा ईमेल पता hgf.45@gmail.com है  ---> 0.9749641418457031
en: My email address is hgf.45@gmail.com  --- te: నా ఇమెయిల్ చిరునామా hgf.45@gmail.com  ---> 0.9708231091499329
en: My email address is hgf.45@gmail.com  --- ta: எனது மின்னஞ்சல் முகவரி hgf.45@gmail.com  ---> 0.975164532661438
en: My email address is hgf.45@gmail.com  --- ja: 私のメールアドレスはhgf.45@gmail.comです  ---> 0.9543834924697876
en: My email address is hgf.45@gmail.com  --- zh-CN: 我的电子邮件地址是 hgf.45@gmail.com  ---> 0.984715461730957
en: My email address is hgf.45@gmail.com  --- fr: Mon adresse e-mail est hgf.45@gmail.com  ---> 0.984859049320221
en: My email address is hgf.45@gmail.com  --- es: Mi dirección de correo electrónico es hgf.45@gmail.com  ---> 0.987299919128418
en: My email address is hgf.45@gmail.com  --- de: Meine E-Mail-Adresse lautet hgf.45@gmail.com  ---> 0.9836717844009399
en: My email

**Sinhala sentece embedding comparison with other language embeddings**

In [ ]:
compare(sentences_set1, forcused_lang="si")
print("----"*30)
compare(sentences_set2, forcused_lang="si")

[INFO] Encoding time 681.6487312316895 ms
torch.Size([11, 768]) torch.Size([768])
en: My email address is hgf.45@gmail.com  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9704842567443848
hi: मेरा ईमेल पता hgf.45@gmail.com है  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9856100082397461
te: నా ఇమెయిల్ చిరునామా hgf.45@gmail.com  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9938134551048279
ta: எனது மின்னஞ்சல் முகவரி hgf.45@gmail.com  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9951666593551636
ja: 私のメールアドレスはhgf.45@gmail.comです  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9621015191078186
zh-CN: 我的电子邮件地址是 hgf.45@gmail.com  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9722886085510254
fr: Mon adresse e-mail est hgf.45@gmail.com  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9785512685775757
es: Mi dirección de correo electrónico es hgf.45@gmail.com  --- si: මගේ ඊමේල් ලිපිනය hgf.45@gmail.com  ---> 0.9824388027191162
de: Meine E-Mail-Adresse lautet hgf.45@g

### Test Word Embeddings

In [ ]:
langs

['en', 'hi', 'te', 'ta', 'ja', 'zh-CN', 'fr', 'es', 'de', 'ru', 'si']

In [ ]:
words_set1 = ["address", "पता", "చిరునామా", "முகவரி", "住所", "地址", "adresse", "dirección", "Adresse", "адрес", "ලිපිනය"]

words_set2 = ["language", "भाषा", "భాష", "மொழி", "言語", "语言", "langue", "idioma", "Sprache", "язык", "භාෂාව"]

In [ ]:
# compare(words_set1, forcused_lang="es")
compare(words_set1, forcused_lang="si")
# print("----"*30)
# compare(words_set2)

[INFO] Encoding time 207.07130432128906 ms
torch.Size([11, 768]) torch.Size([768])
en: address  --- si: ලිපිනය  ---> 0.9555337429046631
hi: पता  --- si: ලිපිනය  ---> 0.8676684498786926
te: చిరునామా  --- si: ලිපිනය  ---> 0.966153621673584
ta: முகவரி  --- si: ලිපිනය  ---> 0.9654062986373901
ja: 住所  --- si: ලිපිනය  ---> 0.8431869745254517
zh-CN: 地址  --- si: ලිපිනය  ---> 0.9238631725311279
fr: adresse  --- si: ලිපිනය  ---> 0.9590113162994385
es: dirección  --- si: ලිපිනය  ---> 0.8250735998153687
de: Adresse  --- si: ලිපිනය  ---> 0.9562503099441528
ru: адрес  --- si: ලිපිනය  ---> 0.9542197585105896


In [ ]:
compare(words_set2, forcused_lang="si")

[INFO] Encoding time 215.35992622375488 ms
torch.Size([11, 768]) torch.Size([768])
en: language  --- si: භාෂාව  ---> 0.9699345827102661
hi: भाषा  --- si: භාෂාව  ---> 0.9870573878288269
te: భాష  --- si: භාෂාව  ---> 0.9886332750320435
ta: மொழி  --- si: භාෂාව  ---> 0.9865323305130005
ja: 言語  --- si: භාෂාව  ---> 0.9443321228027344
zh-CN: 语言  --- si: භාෂාව  ---> 0.932677149772644
fr: langue  --- si: භාෂාව  ---> 0.9604392051696777
es: idioma  --- si: භාෂාව  ---> 0.9059194326400757
de: Sprache  --- si: භාෂාව  ---> 0.9661679267883301
ru: язык  --- si: භාෂාව  ---> 0.9663082361221313


In [ ]:
compare_two("සරත්", "Sarath")
print("----"*30)
compare_two("කසුන්", "Kasun")
print("----"*30)
compare_two("රැජින", "queen")
print("----"*30)

[INFO] Encoding time 98.24156761169434 ms
torch.Size([2, 768]) torch.Size([768])
සරත් -- Sarath ---> tensor([[0.6848]])
------------------------------------------------------------------------------------------------------------------------
[INFO] Encoding time 93.5673713684082 ms
torch.Size([2, 768]) torch.Size([768])
කසුන් -- Kasun ---> tensor([[0.7523]])
------------------------------------------------------------------------------------------------------------------------
[INFO] Encoding time 101.48167610168457 ms
torch.Size([2, 768]) torch.Size([768])
රැජින -- queen ---> tensor([[0.9479]])
------------------------------------------------------------------------------------------------------------------------
